In [ ]:
# 필요한 라이브러리 설치
!pip install pandas
!pip install zipfile36
!pip install IPython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os

zip_file_path = '/content/drive/My Drive/회사업무/사외채널확보/LOCALDATA_ALL_CSV 2.zip'
print("파일 경로:", zip_file_path)

if os.path.exists(zip_file_path):
    print("파일이 존재합니다.")
else:
    print("파일이 존재하지 않습니다. 경로를 다시 확인해주세요.")

파일 경로: /content/drive/My Drive/회사업무/사외채널확보/LOCALDATA_ALL_CSV 2.zip
파일이 존재합니다.


In [ ]:
import zipfile

# ZIP 파일 경로
zip_file_path = '/content/drive/My Drive/회사업무/사외채널확보/LOCALDATA_ALL_CSV 2.zip'

# ZIP 파일에서 파일명을 cp437로 읽기
with zipfile.ZipFile(zip_file_path, 'r') as z:
    print("ZIP 파일 내 파일 목록:")
    for file_name in z.namelist():
        decoded_name = file_name.encode('cp437').decode('euc-kr')  # cp437 -> euc-kr
        print(decoded_name)

ZIP 파일 내 파일 목록:
fulldata_01_01_01_P_병원.csv
fulldata_01_01_02_P_의원.csv
fulldata_01_01_03_P_부속의료기관.csv
fulldata_01_01_04_P_산후조리업.csv
fulldata_01_01_05_P_안전상비의약품판매업소.csv
fulldata_01_01_06_P_약국.csv
fulldata_01_01_07_P_응급환자이송업.csv
fulldata_01_01_08_P_의료법인.csv
fulldata_01_01_10_P_의료유사업.csv
fulldata_01_02_01_P_안경업.csv
fulldata_01_02_02_P_의료기기수리업.csv
fulldata_01_02_03_P_의료기기판매임대업.csv
fulldata_01_02_04_P_치과기공소.csv
fulldata_02_03_01_P_동물병원.csv
fulldata_02_03_02_P_동물약국.csv
fulldata_02_03_03_P_동물용의료용구판매업.csv
fulldata_02_03_04_P_동물용의약품도매상.csv
fulldata_02_03_05_P_동물장묘업.csv
fulldata_02_03_06_P_동물생산업.csv
fulldata_02_03_07_P_동물판매업.csv
fulldata_02_03_08_P_동물수입업.csv
fulldata_02_03_09_P_동물전시업.csv
fulldata_02_03_10_P_동물위탁관리업.csv
fulldata_02_03_11_P_동물미용업.csv
fulldata_02_03_12_P_동물운송업.csv
fulldata_02_04_01_P_가축사육업.csv
fulldata_02_04_02_P_가축인공수정소.csv
fulldata_02_04_03_P_도축업.csv
fulldata_02_04_04_P_부화업.csv
fulldata_02_04_05_P_사료제조업.csv
fulldata_02_04_06_P_종축업.csv
fulldata_03_05_01_P_게임물배급업.csv
fulldata_03_05_

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import zipfile
import chardet
from IPython.display import display, HTML

# ZIP 파일 경로
zip_file_path = '/content/drive/My Drive/회사업무/사외채널확보/LOCALDATA_ALL_CSV 2.zip'

# 특정 키워드 리스트
keywords = ["의원", "약국", "건강기능", "판매업"]
# 특정 주소 필터 리스트
address_filters = ["마포구", "염리동"]

# 파일 인코딩 감지 함수
def detect_encoding(file):
    raw_data = file.read()
    result = chardet.detect(raw_data)
    file.seek(0)  # 파일 포인터를 처음으로 이동
    return result['encoding']

# ZIP 파일에서 CSV 파일 추출 및 키워드 필터링 함수
def extract_and_filter_csv(zip_file_path, keywords, address_filters):
    results = []
    with zipfile.ZipFile(zip_file_path, 'r') as z:
        for file_name in z.namelist():
            # 한글 파일명 디코딩
            decoded_name = file_name.encode('cp437').decode('euc-kr')
            # 파일명에 키워드가 일부라도 포함되어 있는지 확인
            if any(keyword in decoded_name for keyword in keywords):
                # CSV 파일 읽기
                with z.open(file_name) as f:
                    encoding = detect_encoding(f)  # 인코딩 감지
                    try:
                        # 첫 번째 시도: 감지된 인코딩으로 읽기
                        df = pd.read_csv(f, encoding=encoding)
                    except (UnicodeDecodeError, ValueError):
                        f.seek(0)  # 파일 포인터를 처음으로 되돌리기
                        try:
                            # 두 번째 시도: utf-8로 읽기
                            df = pd.read_csv(f, encoding='utf-8')
                        except (UnicodeDecodeError, ValueError) as e:
                            print(f"파일 {decoded_name}에서 인코딩 오류 발생: {e}")
                            continue

                    # 데이터 정제: 공백 열 제거
                    df = df.dropna(axis=1, how='all')  # 모든 값이 NaN인 열 제거
                    df = df.loc[:, (df != '').any(axis=0)]  # 모든 값이 공백인 열 제거

                    # 주소 필터 적용
                    address_column = None
                    if '소재지전체주소' in df.columns:
                        address_column = '소재지전체주소'
                    else:
                        # "주소"라는 키워드가 포함된 첫 번째 열 찾기
                        for col in df.columns:
                            if '주소' in col:
                                address_column = col
                                break

                    if address_column:
                        for addr in address_filters:
                            filtered_df = df[df[address_column].str.contains(addr, na=False)]
                            if not filtered_df.empty:
                                # 네이버 지도 링크 추가
                                filtered_df['네이버지도'] = filtered_df[address_column].apply(
                                    lambda x: f'<a href="https://map.naver.com/v5/search/{x}" target="_blank">지도보기</a>'
                                )

                                # 인허가일자 및 소재지전체주소 기준으로 정렬
                                if '인허가일자' in filtered_df.columns:
                                    filtered_df = filtered_df.sort_values(by=['인허가일자', address_column], ascending=[True, True])
                                elif address_column in filtered_df.columns:
                                    filtered_df = filtered_df.sort_values(by=[address_column], ascending=True)

                                results.append(filtered_df)
                    else:
                        print(f"파일 {decoded_name}에 주소 관련 열이 없습니다.")
    return results

# 데이터프레임 시각화 함수 (반응형 테이블)
def display_responsive_table(dataframes):
    # 모든 결과 데이터프레임을 하나의 데이터프레임으로 병합
    merged_df = pd.concat(dataframes, ignore_index=True)

    # 스타일 및 반응형 디자인
    styles = """
    <style>
        table {
            width: 100%;
            border-collapse: collapse;
        }
        th, td {
            padding: 8px;
            text-align: left;
            border: 1px solid #ddd;
        }
        tr:nth-child(even) {
            background-color: #f2f2f2;
        }
        tr:hover {
            background-color: #ddd;
        }
    </style>
    """
    # HTML 테이블로 표시
    display(HTML(styles + merged_df.to_html(escape=False, index=False)))

# 키워드와 주소로 필터링한 데이터프레임 리스트 가져오기
filtered_dataframes = extract_and_filter_csv(zip_file_path, keywords, address_filters)

# 결과를 반응형 테이블로 표시
if filtered_dataframes:
    display_responsive_table(filtered_dataframes)
else:
    print("검색된 데이터가 없습니다.")

파일 fulldata_01_01_02_P_의원.csv에서 인코딩 오류 발생: 'utf-8' codec can't decode byte 0xb9 in position 1: invalid start byte


<ipython-input-3-a2d9e57253c1>:35: DtypeWarning: Columns (11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, encoding=encoding)
<ipython-input-3-a2d9e57253c1>:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['네이버지도'] = filtered_df[address_column].apply(
<ipython-input-3-a2d9e57253c1>:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['네이버지도'] = filtered_df[address_column].apply(


파일 fulldata_01_01_06_P_약국.csv에서 인코딩 오류 발생: 'utf-8' codec can't decode byte 0xb9 in position 1: invalid start byte


<ipython-input-3-a2d9e57253c1>:35: DtypeWarning: Columns (9,14,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, encoding=encoding)
<ipython-input-3-a2d9e57253c1>:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['네이버지도'] = filtered_df[address_column].apply(
<ipython-input-3-a2d9e57253c1>:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['네이버지도'] = filtered_df[address_column].apply(
<ipython-input-3-a2d9e57253c1>:65: SettingWithCopyWa

파일 fulldata_07_22_03_P_건강기능식품일반판매업.csv에서 인코딩 오류 발생: 'utf-8' codec can't decode byte 0xb9 in position 1: invalid start byte
파일 fulldata_07_22_04_P_축산판매업.csv에서 인코딩 오류 발생: 'utf-8' codec can't decode byte 0xb9 in position 1: invalid start byte


<ipython-input-3-a2d9e57253c1>:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['네이버지도'] = filtered_df[address_column].apply(
<ipython-input-3-a2d9e57253c1>:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['네이버지도'] = filtered_df[address_column].apply(
<ipython-input-3-a2d9e57253c1>:35: DtypeWarning: Columns (11,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, encoding=encoding)
<ipython-input-3-a2d9e57253c1>:65: SettingWithCopyWarn